<a href="https://colab.research.google.com/github/Usama-zia/Project-2/blob/main/project_2_testing_hate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Evaluation on pretrained transformers

#Installing Libraries and Dependencies


In [ ]:
#Installing libraries and Dependencies
!pip install sentencepiece
!pip install transformers
!pip install emoji
!pip install demoji
import emoji
import demoji
from transformers import RobertaTokenizer,TFRobertaForSequenceClassification
from transformers import TFTrainer, TFTrainingArguments
from transformers import XLMTokenizer, TFXLMForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import torch
import tensorflow as tf
import numpy as np
import seaborn as sns
import pandas as pd
import csv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import math
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from textwrap import wrap
from textblob import TextBlob
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

     |████████████████████████████████| 1.2MB 14.8MB/s 
     |████████████████████████████████| 2.1MB 17.9MB/s 
     |████████████████████████████████| 901kB 48.2MB/s 
     |████████████████████████████████| 3.3MB 51.1MB/s 
     |████████████████████████████████| 133kB 17.5MB/s 


#Cloning the Dataset from Github Repository(https://github.com/Usama-zia/Project-2)

In [ ]:
# remove dataset directory if already exists
!rm -rf project-2

# fetch dataset
!git clone https://github.com/Usama-zia/Project-2.git

Cloning into 'Project-2'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 45 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (45/45), done.


#Function for preprocessing Datasets

In [ ]:

def preprocessing_data(data):
  print('============Before Preprocessing=============')

  for index,text in enumerate(data[0][5:10]):
    print('tweet %d:\n'%(index+1),text)
  #removing punctuations
  data[0]=data[0].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
  #removing extra spaces
  data[0]=data[0].apply(lambda x: re.sub(' +',' ',x))
  #removing @user
  data[0] = data[0].str.strip('user')

  #after Preprocessing
  print('============After Preprocessing=============')
  for index,text in enumerate(data[0][0:5]):
    print('tweet %d:\n'%(index+1),text)
  return data

#Some Constant Used for Loading the Dataset

In [ ]:
#folders = [emotion, hate ,sentiment]
emotion = '/content/Project-2/datasets/emotion'
hate = '/content/Project-2/datasets/hate'
sentiment = '/content/Project-2/datasets/sentiment'
train_text = 'train_text.txt'
train_label = 'train_labels.txt'
test_text = 'test_text.txt'
test_label = 'test_labels.txt'
val_text = 'val_text.txt'
val_label = 'val_labels.txt'
maps = 'mapping.txt'

#Function for Loading Datasets

In [ ]:
data_set=emotion #variable is declared outsids function so os.path.join does not through an error on compile
def load_data(data_set):
  #for loading train text and labels
  with open(os.path.join(data_set, train_text)) as f:
    train_texts = pd.read_csv(f, sep="\n", header=None)
    with open(os.path.join(data_set,train_label)) as f:
      train_labels = pd.read_csv(f, sep="\n", header=None)

  with open(os.path.join(data_set, val_text)) as f:
    val_texts = pd.read_csv(f, sep="\n", header=None)
    with open(os.path.join(data_set, val_label)) as f:
      val_labels = pd.read_csv(f, sep="\n", header=None)

  with open(os.path.join(data_set, test_text)) as f:
    test_texts = pd.read_csv(f, sep="\n", header=None)
    with open(os.path.join(data_set, test_label)) as f:
      test_labels = pd.read_csv(f, sep="\n", header=None)
  return train_texts, train_labels,val_texts,val_labels,test_texts,test_labels

#Loading and preprocessing hate dataset

In [ ]:
train_texts,train_labels,val_texts,val_labels,test_texts,test_labels=load_data(hate)
train_data = preprocessing_data(train_texts)
val_data = preprocessing_data(val_texts)
test_data = preprocessing_data(test_texts)

============Before Preprocessing=============
tweet 1:
 Me flirting- So tell me about your father... 
tweet 2:
 The Philippine Catholic bishops' work for migrant workers should focus on families who are "paying the great... 
tweet 3:
 I AM NOT GOING AFTER YOUR EX BF YOU LIEING SACK OF SHIT ! I'm done with you dude that's why I dumped your ass cause your a lieing 😂😡 bitch 
tweet 4:
 When cuffin season is finally over 
tweet 5:
 Send home migrants not in need of protection, Peter Dutton tells UN, HEY DUTTON HOW ABOUT THE ONES THAT HAVE STAYED AND NOT LEFT THE COUNTRY WHEN THEY SHOULD OVERSTAYERS ? WHY DONT YOU GO AND ROUND ALL THEM UP ?  
============After Preprocessing=============
tweet 1:
  nice new signage Are you not concerned by Beatlemania style hysterical crowds crongregating on you… 
tweet 2:
 A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎 
tweet 3:
  user real talk do you have eyes or were they gouged out by a rapefugee 
tw

#Tokenization


In [ ]:
def compute_metrics(pred,labels):
    labels = labels
    preds = pred
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-hate"
pred=[]
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL)
for text in test_data[0]:
  encoded_input = tokenizer(text, return_tensors='pt')
  pred.append(model(**encoded_input)[0].argmax().item())

#Evaluation

In [ ]:
labels=[]
for l in test_labels[0]:
  labels.append(l)

In [ ]:
result=compute_metrics(pred,labels)

In [ ]:
print(result)